In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import operator
import numbers

In [2]:
full_data = pd.read_csv('../../../Data/07_FinalProject/Most-Recent-Cohorts-All-Data-Elements.csv')

/anaconda3/envs/NLP/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,9,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1008,1009,1010,1011,1014,1015,1016,1017,1018,1019,1021,1022,1023,1027,1028,1029,1030,1031,1032,1034,1035,1036,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1065,1066,1067,1068,1069,1070,1071,1073,1074,1075,1076,1078,1079,1080,1081,1082,1083,1084,1086,1087,1088,1089,1091,1092,1093,1094,1095,1096,1097,1099,1100,1101,1102,1104,1105,1106,1107,1108,1109,1110,1112,1113,1114,1115,1118,1119,1121,1122,1123,1125,1127,1128,1131,1132,1134,1135,1136,1138,1140,1141,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1170,1171,1172,1173,1174,1175,1177,1178,1179,1180,1183,1184,1185,1186,1187,1188,1190,1192,1196,1199,1200,1201,1209,1212,1213,1214,1222,1223,1224,1225,1226,1227,1235,1236

In [3]:
def normalizeColumn(column, data):
    if isinstance(data[column][0], numbers.Number):
        return [float(i)/sum(data[column].values) for i in data[column].values]
    else:
        return data[column]

def normalizeData(data):
    for column in data.columns:
        data[column] = normalizeColumn(column, data)
    return data

def cleanColumn(column, data):
    if type(data[column][0]) == type('.'):
        data[column][data[column] == 'PrivacySuppressed'] = np.nan
    return data[column]

def cleanData(data):
    for column in data.columns:
        data[column] = cleanColumn(column, data)
    return data

def imputeValue(column, data):
    column_data = pd.to_numeric(data[column])
    column_mean = column_data.mean()
    for i, row in enumerate(column_data):
        if np.isnan(row):
            column_data.iloc[i] = column_mean
    return column_data

def imputeNumeric(columns, data):
    for column in columns:
        data[column] = imputeValue(column, data)
    return data

def processCategorical(columns, data):
    for column in columns:
        data[column] = data[column].astype('category')
    return data

In [4]:
def calculateCostEarning(data):
    cost_a = list(data['COSTT4_A'])
    cost_p = list(data['COSTT4_P'])
    cost = []
    for i in range(len(cost_a)):
        if not np.isnan(cost_a[i]):
            if not np.isnan(cost_p[i]):
                raise Exception('No school is like this')
            cost.append(cost_a[i])
        else:
            cost.append(cost_p[i])
    earning = data['MD_EARN_WNE_P6'] 
    return pd.DataFrame({'cost':cost, 'earning':earning})

def calculateC100(data):
    target_C100 = []
    C100_4 = data['C100_4']
    C100_L4 = data['C100_L4']
    data = data.drop(['C100_4', 'C100_L4'], 1)
    for i in range(len(C100_4)):
        if not np.isnan(C100_4[i]):
            if not np.isnan(C100_L4[i]):
                raise Exception('No school is like this')
            target_C100.append(C100_4[i])
        else:
            target_C100.append(C100_L4[i])
    data['C100_4'] = target_C100
    return data

def fillMean(data):
    data['AVGFACSAL'].fillna((data['AVGFACSAL'].mean()), inplace=True)
    data['UG'].fillna((data['UG'].mean()), inplace=True)
    data['PFTFAC'].fillna((data['PFTFAC'].mean()), inplace=True)
    data['UGDS_MEN'].fillna((data['UGDS_MEN'].mean()), inplace=True)
    data['UGDS_WOMEN'].fillna((data['UGDS_WOMEN'].mean()), inplace=True)
    return data

def fillIncome(data):
    income = {1: 44765, 2:73355, 4: 51492, 5: 41995, 6:64500, 8:63909, 9:71346,
              10:61255, 11:75628,12: 49426, 13: 51244, 15: 73486, 16: 48275, 
              17:59588, 18: 50532, 19:54736, 20: 53906, 21: 45215, 22:45727,
             23:51494, 24:75847, 25:70628 , 26:51084, 27:63488,
             28: 40593, 29:50238, 30:49509, 31:54996, 32:52431,
             33: 70303, 34:72222, 35: 45382, 36:60850, 37:47830,
             38: 60577, 39:51075, 40:48568, 41:54148, 42:55702,
             44: 58073, 45:47238, 46: 53017, 47: 47275, 48:55653,
             49: 62912, 50: 56990, 51: 66262, 53:64129, 54:42019,
             55: 55638, 56: 60214}
    data['ST_FIPS'] = data['ST_FIPS'].astype(float)
    data['MEDIAN_HH_INC'].fillna(data.ST_FIPS.map(income), inplace=True)
    data['MEDIAN_HH_INC'] = data['MEDIAN_HH_INC'].astype(float)
    data['MEDIAN_HH_INC'].fillna(data['MEDIAN_HH_INC'].mean(), inplace=True)
    return data

##HISP: Hispanic students rose from 4 percent to 17 percent 
# API: Asian/Pacific Islander students rose from 2 percent to 7 percent
# Black: The percentage of Black students increased from 10 percent in 1976 to 14 percent in 2015
# White: 58%
def fillRace(data):
    data['UG_NRA'].fillna(0.01, inplace=True)
    data['UG_WHITENH'].fillna(0.58, inplace=True)
    data['UG_BLACKNH'].fillna(0.14, inplace=True)
    data['UG_API'].fillna(0.035, inplace=True)
    data['UG_AIANOLD'].fillna(0.035, inplace=True)
    data['UG_HISPOLD'].fillna(0.17, inplace=True)
    data['UG_UNKN'].fillna((1 - data['UG_NRA'] - 
                                data['UG_WHITENH'] - 
                                data['UG_BLACKNH'] - 
                                data['UG_API'] - 
                                data['UG_AIANOLD'] - 
                                data['UG_HISPOLD']),inplace=True)
    return data

def fillUnemp(data):
    unemp = {1: 3.8, 2:7.3, 4: 4.9, 5: 3.8, 6:4.3, 8:3.0, 9:4.5, 
             10:4.3, 11:5.6,12: 3.9, 13: 4.4, 15: 2.1, 16: 2.9, 17:4.6, 
             18: 3.2, 19:2.8, 20: 3.4, 21: 4.0, 22:4.4,
             23:2.7, 24:4.3, 25:3.5 , 26:4.7, 27:3.2,
             28: 4.5, 29:3.6, 30:4.1, 31:2.8, 32:4.9,
             33: 2.6, 34:4.6, 35: 5.6, 36:4.6, 37:4.5,
             38: 2.6, 39:4.4, 40:4.0, 41:4.1, 42:4.8,
             44: 4.5, 45:4.4, 46: 3.4, 47: 3.4, 48:4.0,
             49: 3.1, 50: 2.8, 51: 3.4, 53:4.8, 54:5.4,
             55: 2.9, 56: 3.9, 60:18, 66:4.4, 69:11.2, 72:8.8, 78:9.2}
    data['ST_FIPS'] = data['ST_FIPS'].astype(float)
    data['UNEMP_RATE'].fillna(data.ST_FIPS.map(unemp), inplace=True)
    data['UNEMP_RATE'] = data['UNEMP_RATE'].astype(float)
    data['UNEMP_RATE'].fillna(data['UNEMP_RATE'].mean(), inplace=True)
    return data

def calculateROI(data):
    data['cost'] = data['cost'].astype('int64')
    data['earning'] = data['earning'].astype('int64')
    data['ROI'] = (data['earning'])/data['cost']
    data.drop(['cost', 'earning'], 1, inplace=True)
    return data

## Feature Selection

In [ ]:
features = []

admin_features = ['ADM_RATE', 'SAT_AVG']
#repay_features = ['COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT', 'COMPL_RPY_7YR_RT', 'RPY_1YR_N', 'RPY_3YR_N', 'RPY_5YR_N', 'RPY_7YR_N']
complete_features = ['D100_4', 'D100_L4', 'C100_4', 'C100_L4']
#aid_features = ['GRAD_DEBT_MDN_SUPP', 'DEBT_MDN_SUPP', 'CUML_DEBT_N', 'GRAD_DEBT_MDN', 'DEBT_N', 'GRAD_DEBT_N']
root_features = ['LATITUDE', 'LONGITUDE']
school_features = ['REGION', 'LOCALE', 'ACCREDAGENCY', 'SCH_DEG', 'MAIN', 'NUMBRANCH', 
                   'HIGHDEG', 'CONTROL', 'ST_FIPS', 'AVGFACSAL', 'PFTFAC','ICLEVEL']
student_features = ['UG', 'UG_NRA', 'UG_UNKN', 'UG_WHITENH', 'UG_BLACKNH', 'UG_API', 'UG_AIANOLD', 
                    'UG_HISPOLD', 'APPL_SCH_PCT_GE2', 'APPL_SCH_PCT_GE3', 'APPL_SCH_PCT_GE4', 'APPL_SCH_PCT_GE5', 
                    'MEDIAN_HH_INC', 'UNEMP_RATE', 'UGDS_MEN', 'UGDS_WOMEN','UGNONDS','GRADS']

features.extend(admin_features)
#features.extend(repay_features)
features.extend(complete_features)
#features.extend(aid_features)
features.extend(root_features)
features.extend(school_features)
features.extend(student_features)

# We drop these variables because over 50% data in those variables are missing.
features_drop = [#'ADM_RATE', 'SAT_AVG', 'RPY_1YR_N', 'RPY_3YR_N', 'RPY_5YR_N', 'RPY_7YR_N',
                'D100_4', 'D100_L4', 'APPL_SCH_PCT_GE2', 'APPL_SCH_PCT_GE3', 'APPL_SCH_PCT_GE4', 
                 'APPL_SCH_PCT_GE5', 'UGNONDS','GRADS', 'ACCREDAGENCY',
    #'GRAD_DEBT_MDN_SUPP', 'CUML_DEBT_N',  
    'CONTROL'
]

data = full_data[features]
data = data.drop(features_drop, 1)
cost_earning_df = calculateCostEarning(full_data)
data['cost'] = cost_earning_df['cost']
data['earning'] = cost_earning_df['earning']
data = cleanData(data)

/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Fill NA values

In [ ]:
numeric_columns = [#'COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT','COMPL_RPY_7YR_RT', 'DEBT_MDN_SUPP', 'GRAD_DEBT_MDN', 'DEBT_N', 'GRAD_DEBT_N', 
    'LATITUDE', 'LONGITUDE', 'NUMBRANCH', 'C100_4']

categorical_column = ['REGION', 'LOCALE', 'SCH_DEG', 'MAIN', 'HIGHDEG', 'ST_FIPS', 'ICLEVEL']#, 'CONTROL']

Y_column = ['earning', 'cost']

data = calculateC100(data)
data = imputeNumeric(numeric_columns, data)
data = fillMean(data)
data = fillRace(data)
data = fillIncome(data)
data = fillUnemp(data)
data = processCategorical(categorical_column, data)
data.dropna(inplace=True)
data = calculateROI(data)

## Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

X = data.drop('ROI', 1)
y = data['ROI']
linear = LinearRegression()
linear = linear.fit(X, y)
coef = linear.coef_

feature_dict = dict(zip(data.columns, coef))
feature_dict = sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=1)
feature_dict

[('UG_AIANOLD', 42.236918974779464),
 ('UG_HISPOLD', 41.54238516855695),
 ('UG_API', 41.29448697157282),
 ('UG_WHITENH', 41.07676395150383),
 ('UG_BLACKNH', 40.884113056165),
 ('UG_UNKN', 40.39352660479426),
 ('UG_NRA', 39.05106325549261),
 ('ICLEVEL', 0.34745090711101084),
 ('PFTFAC', 0.06926663134545404),
 ('HIGHDEG', 0.06265812756961989),
 ('ADM_RATE', 0.027721241403698547),
 ('LATITUDE', 0.0015110142457203319),
 ('ST_FIPS', 0.0007116712587853755),
 ('SAT_AVG', 0.00023069909662947342),
 ('LOCALE', 6.465445170345758e-05),
 ('AVGFACSAL', 5.1409472726589836e-05),
 ('UG', 2.543272878398211e-05),
 ('MEDIAN_HH_INC', -8.386895060977594e-06),
 ('LONGITUDE', -0.0050426082626804595),
 ('NUMBRANCH', -0.01591807243524326),
 ('UNEMP_RATE', -0.04907594107393445),
 ('REGION', -0.04959114716960523),
 ('SCH_DEG', -0.397048465240461),
 ('MAIN', -0.69060886099352),
 ('C100_4', -1.1074733258141374),
 ('UGDS_MEN', -1962.144644204354),
 ('UGDS_WOMEN', -1962.249786172121)]

## Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

X = data.drop('ROI', 1)
y = data['ROI']
gbm = GradientBoostingRegressor()
gbm = gbm.fit(X, y)
importance = gbm.feature_importances_

feature_dict = dict(zip(data.columns, importance))
feature_dict = sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=1)
feature_dict

[('UG', 0.2646019075378572),
 ('C100_4', 0.166084834479933),
 ('REGION', 0.13327748154396898),
 ('MAIN', 0.0640217538175726),
 ('MEDIAN_HH_INC', 0.04394368935811733),
 ('UG_NRA', 0.036530523760220125),
 ('LONGITUDE', 0.03478276741763144),
 ('UGDS_MEN', 0.03383207648179556),
 ('AVGFACSAL', 0.02887018806984351),
 ('UG_UNKN', 0.02603771237868174),
 ('UG_AIANOLD', 0.025543728315320202),
 ('UGDS_WOMEN', 0.024665748452932505),
 ('UG_BLACKNH', 0.022303229316099925),
 ('SCH_DEG', 0.02180260377745126),
 ('UG_API', 0.01158449828664949),
 ('LATITUDE', 0.008636758026047162),
 ('UNEMP_RATE', 0.008181305474910916),
 ('UG_WHITENH', 0.008129119847147552),
 ('PFTFAC', 0.00792078723844429),
 ('ADM_RATE', 0.007180615543426526),
 ('HIGHDEG', 0.005994017233357978),
 ('SAT_AVG', 0.0056016424365408065),
 ('UG_HISPOLD', 0.00495781543157282),
 ('ST_FIPS', 0.002540752361699047),
 ('NUMBRANCH', 0.001480820789017637),
 ('LOCALE', 0.0008326502646306702),
 ('ICLEVEL', 0.0006609723591298179)]

In [ ]:
import lightgbm as lgb

X = data.drop('ROI', 1)
y = data['ROI']
gbm = lgb.LGBMRegressor()
gbm = gbm.fit(X, y)
importance = gbm.feature_importances_